# Part 1

Here, you encounter a pair of dueling generators. The generators, called generator A and generator B, are trying to agree on a sequence of numbers. However, one of them is malfunctioning, and so the sequences don't always match.

As they do this, a judge waits for each of them to generate its next value, compares the lowest 16 bits of both values, and keeps track of the number of times those parts of the values match.

The generators both work on the same principle. To create its next value, a generator will take the previous value it produced, multiply it by a factor (generator A uses 16807; generator B uses 48271), and then keep the remainder of dividing that resulting product by `2147483647`. That final remainder is the value it produces next.

To calculate each generator's first value, it instead uses a specific starting value as its "previous value" (as listed in your puzzle input).

For example, suppose that for starting values, generator A uses 65, while generator B uses 8921. Then, the first five pairs of generated values are:

```
--Gen. A--  --Gen. B--
   1092455   430625591
1181022009  1233683848
 245556042  1431495498
1744312007   137874439
1352636452   285222916
```

In binary, these pairs are (with generator A's value first in each pair):

```
00000000000100001010101101100111
00011001101010101101001100110111

01000110011001001111011100111001
01001001100010001000010110001000

00001110101000101110001101001010
01010101010100101110001101001010

01100111111110000001011011000111
00001000001101111100110000000111

01010000100111111001100000100100
00010001000000000010100000000100
```

Here, you can see that the lowest (here, rightmost) 16 bits of the third value match: `1110001101001010`. Because of this one match, after processing these five pairs, the judge would have added only 1 to its total.

To get a significant sample, the judge would like to consider 40 million pairs. (In the example above, the judge would eventually find a total of 588 pairs that match in their lowest 16 bits.)

After 40 million pairs, what is the judge's final count?

```
Generator A starts with 512
Generator B starts with 191
```

## Plain Python

In [13]:
import itertools

In [21]:
genA_test = 65
genB_test = 8921

genA_puzzle = 512
genB_puzzle = 191

genA_factor = 16807
genB_factor = 48271

In [14]:
def gen(start, factor):
    result = start
    while True:
        result = (result * factor) % 2147483647
        yield result

In [15]:
def genA(start):
    yield from gen(start, 16807)
    
def genB(start):
    yield from gen(start, 48271)

In [16]:
assert list(itertools.islice(genA(genA_test), 5)) == [1092455, 1181022009, 245556042, 1744312007, 1352636452]
assert list(itertools.islice(genB(genB_test), 5)) == [430625591, 1233683848, 1431495498, 137874439, 285222916]

In [17]:
def bin16(x):
    return bin(x)[2:][-16:].zfill(16)

In [18]:
def judge(gA, gB, num):
    gA = itertools.islice(gA, num)
    gB = itertools.islice(gB, num)
    return sum(bin16(val1) == bin16(val2) for val1, val2 in zip(gA, gB))

In [8]:
result = judge(genA(genA_test), genB(genB_test), 40000000)
assert result == 588, result

In [9]:
%time judge(genA(genA_puzzle), genB(genB_puzzle), 40000000)

CPU times: user 1min 54s, sys: 909 ms, total: 1min 55s
Wall time: 2min


567

## Cython

In [3]:
import cytoolz as tz

In [1]:
%load_ext cython

In [67]:
%%cython
cdef class cygen:
    cdef unsigned long result, factor
    
    def __cinit__(self, unsigned long result, unsigned long factor):
        self.result = result
        self.factor = factor
    
    def __iter__(self):
        return self
    
    def __next__(self):
        cdef unsigned long _, val
        val = (self.result * self.factor) % 2147483647
        self.result = val
        return val

cpdef str cybin16(unsigned long x):
    return bin(x)[2:][-16:].zfill(16)

cpdef int cyjudge(object gA, object gB):
    cdef int count = 0
    cdef unsigned long val1, val2
    for val1, val2 in zip(gA, gB):
        if cybin16(val1) == cybin16(val2):
            count += 1
    return count

In [60]:
%timeit list(itertools.islice(genA(genA_test), 1000))

276 µs ± 2.42 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [61]:
%timeit list(tz.take(1000, cygen(genA_test, 16807)))

34.3 µs ± 1.61 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [62]:
assert cybin16(100) == bin16(100)

In [63]:
%timeit bin16(123)

585 ns ± 9.91 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [64]:
%timeit cybin16(123)

465 ns ± 12.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [65]:
assert list(tz.take(5, cygen(genA_test, genA_factor))) == [1092455, 1181022009, 245556042, 1744312007, 1352636452]
assert list(tz.take(5, cygen(genB_test, genB_factor))) == [430625591, 1233683848, 1431495498, 137874439, 285222916]

In [68]:
%%time
num = 40000000
result = sum(cybin16(val1) == cybin16(val2) 
    for val1, val2 in zip(tz.take(num, cygen(genA_test, genA_factor)), tz.take(num, cygen(genB_test, genB_factor))))
assert result == 588, result

CPU times: user 50.4 s, sys: 123 ms, total: 50.6 s
Wall time: 50.7 s


In [69]:
%%time
num = 40000000
sum(cybin16(val1) == cybin16(val2) 
    for val1, val2 in zip(tz.take(num, cygen(genA_puzzle, genA_factor)), tz.take(num, cygen(genB_puzzle, genB_factor))))

CPU times: user 51.3 s, sys: 134 ms, total: 51.4 s
Wall time: 51.6 s


In [70]:
%%time
num = 40000000
result = cyjudge(tz.take(num, cygen(genA_test, genA_factor)), tz.take(num, cygen(genB_test, genB_factor)))
assert result == 588, result

CPU times: user 40.3 s, sys: 245 ms, total: 40.6 s
Wall time: 40.5 s


In [71]:
%%time
num = 40000000
result = cyjudge(tz.take(num, cygen(genA_puzzle, genA_factor)), tz.take(num, cygen(genB_puzzle, genB_factor)))
print(result)

567
CPU times: user 41.9 s, sys: 342 ms, total: 42.3 s
Wall time: 42.3 s


In [72]:
%%cython
cdef str cybin16(unsigned long x):
    return bin(x)[2:][-16:].zfill(16)

cpdef int cyjudge2():
    cdef int count = 0
    cdef unsigned long _
    cdef unsigned long mod_factor = 2147483647
    cdef unsigned long a_factor = 16807
    cdef unsigned long b_factor = 48271
    cdef unsigned long a_result = 512
    cdef unsigned long b_result = 191
    
    for _ in range(40000000):
        a_result = (a_result * a_factor) % mod_factor
        b_result = (b_result * b_factor) % mod_factor
        if cybin16(a_result) == cybin16(b_result):
            count += 1
    
    return count

In [73]:
%time cyjudge2()

CPU times: user 38.2 s, sys: 284 ms, total: 38.5 s
Wall time: 38.5 s


567

# Part 2

In the interest of trying to align a little better, the generators get more picky about the numbers they actually give to the judge.

They still generate values in the same way, but now they only hand a value to the judge when it meets their criteria:

```
Generator A looks for values that are multiples of 4.
Generator B looks for values that are multiples of 8.
```

Each generator functions completely independently: they both go through values entirely on their own, only occasionally handing an acceptable value to the judge, and otherwise working through the same sequence of values as before until they find one.

The judge still waits for each generator to provide it with a value before comparing them (using the same comparison method as before). It keeps track of the order it receives values; the first values from each generator are compared, then the second values from each generator, then the third values, and so on.

Using the example starting values given above, the generators now produce the following first five values each:

```
--Gen. A--  --Gen. B--
1352636452  1233683848
1992081072   862516352
 530830436  1159784568
1980017072  1616057672
 740335192   412269392
```

These values have the following corresponding binary values:

```
01010000100111111001100000100100
01001001100010001000010110001000

01110110101111001011111010110000
00110011011010001111010010000000

00011111101000111101010001100100
01000101001000001110100001111000

01110110000001001010100110110000
01100000010100110001010101001000

00101100001000001001111001011000
00011000100100101011101101010000
```

Unfortunately, even though this change makes more bits similar on average, none of these values' lowest 16 bits match. Now, it's not until the 1056th pair that the judge finds the first match:

```
--Gen. A--  --Gen. B--
1023762912   896885216

00111101000001010110000111100000
00110101011101010110000111100000
```

This change makes the generators much slower, and the judge is getting impatient; it is now only willing to consider 5 million pairs. (Using the values from the example above, after five million pairs, the judge would eventually find a total of 309 pairs that match in their lowest 16 bits.)

After 5 million pairs, but using this new generator logic, what is the judge's final count?

In [10]:
def genA2(start):
    for x in gen(start, 16807):
        if x % 4 == 0:
            yield x
    
def genB2(start):
    for x in gen(start, 48271):
        if x % 8 == 0:
            yield x

In [11]:
result = judge(genA2(genA_test), genB2(genB_test), 5000000)
assert result == 309, result

In [12]:
judge(genA2(genA_puzzle), genB2(genB_puzzle), 5000000)

323